<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with a custom metrics provider

This notebook demonstrates how to configure the custom monitor and custom metrics deployment by using IBM Watson OpenScale. It requires Python 3.10 and service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning

  
## Contents

This notebook contains the following parts:

  1. [Set up your environment.](#setup)
  1. [Create the custom metrics provider - python function.](#provider)
  1. [Register the custom metrics provider and create a deployment.](#deployment)
  1. [Configure Watson OpenScale](#config)
  1. [Create the integrated system for the custom metrics provider.](#custom)
  1. [Set up the custom monitor definition and instance.](#instance)


## 1. Set up your environment. <a name="setup"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Install the  `ibm-watson-machine-learning` and `ibm-watson-openscale` packages.

In [1]:
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

### Credentials for IBM Cloud Pak for Data
To authenticate, in the following code boxes, replace the sample data with your own credentials. Get the information from your system administrator or through the Cloud Pak for Data dashboard.


### Obtaining your Watson OpenScale credentials

You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'` Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
############################################################################################
# Paste your Watson OpenScale credentials into the following section and then run this cell.
############################################################################################
WOS_CREDENTIALS = {
    "url": "xxxxx",
    "username": "xxxxx",
    "apikey": "xxxxx"
}


### Enter your Watson OpenScale GUID.

For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.


In [4]:
#Update your Watson OpenScale datamart id.
WOS_GUID="00000000-0000-0000-0000-000000000000"

In [5]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
#WML_CREDENTIALS['version']='4.0'
WML_CREDENTIALS

{'url': 'https://cpd-cpd.scotiabank-watsonx-75a08960998068286ff1d6f41bddfabb-0000.ca-tor.containers.appdomain.cloud',
 'username': 'cpadmin',
 'apikey': 'gt6bW9bX5Q5O6CP1KnV94y1GQjQ9dRJI4zkQAesr',
 'instance_id': 'openshift'}

### Define the Watson OpenScale subscription to which the custom metrics have to be sent.

Create a subscription from the Watson Openscale UI or SDK to configure custom metrics. You can configure custom metrics for the subscriptions that have predefined monitors, such as fairness, quality, or drift or without predefined monitors.

In [6]:
####################################################################
# Paste your Subscription in the following field and then run this cell.
####################################################################

subscription_id = "xxxxx"

### Python function details

In [7]:
PYTHON_FUNCTION_NAME = 'Human Rating Metrics Provider Function'
DEPLOYMENT_NAME = 'Human Rating Metrics Provider Deployment'



### OpenScale Custom Metrics Provider name

In [8]:
CUSTOM_METRICS_PROVIDER_NAME = "Human Rating Metrics Provider"



### OpenSale Custom Monitor name

In [9]:
###################################################################
# UPDATE your custom monitor name in the following field and then run this cell.
####################################################################
CUSTOM_MONITOR_NAME = 'Human Rating Performance' 


## 2. Create the custom metrics provider - Python function. <a name="provider"></a>

The Python function receives the required variables, such as the `datamart_id`, `monitor_instance_id`, `monitor_id`, `monitor_instance_parameters` and `subscription_id` from the Watson OpenScale service when it is invoked by the custom monitor. 

In the Python function, add your own logic to compute the custom metrics in the `get_metrics` method, publish the metrics to the Watson Openscale service and update the status of the run to the `finished` state in the custom monitor instance.

Update the `WOS_CREDENTIALS` in the Python function. 

In [64]:
#wml_python_function
parms = {
        "url": WOS_CREDENTIALS["url"],
        "username": WOS_CREDENTIALS["username"],
        "apikey": WOS_CREDENTIALS["apikey"]
    }
def custom_metrics_provider(parms = parms):
    
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid    
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    
    
    # Get the access token
    def get_access_token():
        url = '{}/icp4d-api/v1/authorize'.format(parms['url'])
        payload = {
            'username': parms['username'],
            'api_key': parms['apikey']
        }
        response = requests.post(url, headers=headers, json=payload, verify=False)
        json_data = response.json()
        access_token = json_data['token']
        return access_token
    
    def get_feedback_dataset_id(access_token, data_mart_id, subscription_id):
        headers["Authorization"] = "Bearer {}".format(access_token)
        DATASETS_URL =  parms["url"] + "/openscale/{0}/v2/data_sets?target.target_id={1}&target.target_type=subscription&type=feedback".format(data_mart_id, subscription_id)
        response = requests.get(DATASETS_URL, headers=headers, verify=False)
        json_data = response.json()
        feedback_dataset_id = None
        if "data_sets" in json_data and len(json_data["data_sets"]) > 0:
            feedback_dataset_id = json_data["data_sets"][0]["metadata"]["id"]
        
        return feedback_dataset_id
    
    def get_feedback_data(access_token, data_mart_id, feedback_dataset_id):
        json_data = None
        if feedback_dataset_id is not None:
            headers["Authorization"] = "Bearer {}".format(access_token)
            DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&format=list".format(data_mart_id, feedback_dataset_id, 100)
            response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
            json_data = response.json()
            return json_data
    
    #Update the run status to Finished in the custom monitor instance
    def update_monitor_instance(base_url, access_token, custom_monitor_instance_id, payload):
        monitor_instance_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '?update_metadata_only=true'
        
        patch_payload  = [
            {
                "op": "replace",
                "path": "/parameters",
                "value": payload
            }
        ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_instance_url, headers=headers, json = patch_payload, verify=False)
        monitor_response = response.json()
        return response.status_code, monitor_response
    
    #Add your code to compute the custom metrics. 
    def get_metrics(access_token, data_mart_id, subscription_id):
        #Add the logic here to compute the metrics. Use the below metric names while creating the custom monitor definition
        feedback_dataset_id = get_feedback_dataset_id(access_token, data_mart_id, subscription_id)
        json_data = get_feedback_data(access_token, data_mart_id, feedback_dataset_id)
        gender_less40_fav_prediction_ratio = 6.4
        #if json_data is not None:
        #    fields = json_data['records'][0]['fields']
        #    values = json_data['records'][0]['values']
        #    import pandas as pd
        #    feedback_data = pd.DataFrame(values, columns = fields)
        #    female_less40_fav_prediction = len(feedback_data.query('Sex == \'female\' & Age <= 40 & Risk == \'No Risk\''))
        #    male_less40_fav_prediction = len(feedback_data.query('Sex == \'male\' & Age <= 40 & Risk == \'No Risk\''))
        #    gender_less40_fav_prediction_ratio = female_less40_fav_prediction / male_less40_fav_prediction

        metrics = {"human_rating": gender_less40_fav_prediction_ratio, "region": "us-south"}
        
        return metrics
        
        
    # Publishes the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp):
        # Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = get_metrics(access_token, data_mart_id, subscription_id)
        measurements_payload = [
                  {
                    "timestamp": timestamp,
                    "run_id": custom_monitoring_run_id,
                    "metrics": [custom_metrics]
                  }
                ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()
     
        return response.status_code, published_measurement
        
    
    def publish( input_data ):
        import datetime
        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
        payload = input_data.get("input_data")[0].get("values")
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']

        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()
        
        published_measurements = []
        error_msgs = []
        
        custom_monitoring_run_id = custom_monitor_instance_params["run_details"]["run_id"]
        try:
            last_run_time = custom_monitor_instance_params.get("last_run_time")
            max_records = custom_monitor_instance_params.get("max_records")
            min_records = custom_monitor_instance_params.get("min_records")

            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp)
            if int(status_code) in [200, 201, 202]:
                custom_monitor_instance_params["run_details"]["run_status"] = "finished"
                published_measurements.append(published_measurement)
            else:
                custom_monitor_instance_params["run_details"]["run_status"] = "error"
                custom_monitor_instance_params["run_details"]["run_error_msg"] = published_measurement
                error_msgs.append(published_measurement)
                    
            custom_monitor_instance_params["last_run_time"] = timestamp
            status_code, response = update_monitor_instance(base_url, access_token, custom_monitor_instance_id, custom_monitor_instance_params)
            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)
                
        except Exception as ex:
            error_msgs.append(str(ex))
        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{ 
                    "values" : published_measurements
                }]

            }
        else:
            response_payload = {
                "predictions":[],
                "errors": error_msgs
            }
        
        return response_payload
        
    return publish
    

## 3. Register the custom metrics provider and create a deployment. <a name="deployment"></a>

In [65]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.360'

In [66]:
wml_client.spaces.list(limit=10)

------------------------------------  -----------------------------------  ------------------------
ID                                    NAME                                 CREATED
4a35a9e0-d33b-40ca-8551-9d77282dc917  Fraud model DS                       2024-07-10T19:48:21.910Z
ab5e72e2-80b1-4917-b0f8-b11b9c487c22  Custom metrics deployment space      2024-07-09T19:58:34.700Z
c4ddf9fe-0652-4a10-a729-1402af27d379  GCP VertexAI - LLM - Pre-production  2024-07-09T15:01:42.196Z
6c989051-ee3e-4bc0-9eb5-d57ae8ad7aa6  GCP VertexAI - LLM - Production      2024-07-08T15:38:31.500Z
------------------------------------  -----------------------------------  ------------------------


,ID,NAME,CREATED
0,4a35a9e0-d33b-40ca-8551-9d77282dc917,Fraud model DS,2024-07-10T19:48:21.910Z
1,ab5e72e2-80b1-4917-b0f8-b11b9c487c22,Custom metrics deployment space,2024-07-09T19:58:34.700Z
2,c4ddf9fe-0652-4a10-a729-1402af27d379,GCP VertexAI - LLM - Pre-production,2024-07-09T15:01:42.196Z
3,6c989051-ee3e-4bc0-9eb5-d57ae8ad7aa6,GCP VertexAI - LLM - Production,2024-07-08T15:38:31.500Z


In [67]:
space_id = "ab5e72e2-80b1-4917-b0f8-b11b9c487c22" #update your space id
wml_client.set.default_space(space_id)

'SUCCESS'

### Remove existing function and deployment.

In [69]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)

wml_client.repository.list_functions()

------------------------------------  ----------------------------------------  ------------------------  ------  ----------  ----------------
GUID                                  NAME                                      CREATED                   TYPE    SPEC_STATE  SPEC_REPLACEMENT
16cd38f5-a80d-439e-b379-1fb518d82563  Human Rating Metrics Provider Function    2024-07-19T15:21:33.002Z  python  supported
12cc063d-01bd-4a40-82cd-4db68624f24c  LLM Metrics Provider Function             2024-07-19T14:26:51.002Z  python  supported
98345c71-f4b2-4fed-bbc7-e125374e5e4a  GenAI Metrics Provider Function           2024-07-19T13:38:36.002Z  python  supported
760f0e2d-9c98-4716-a30f-a00477b62aee  test Metrics Provider Function            2024-07-19T03:51:08.002Z  python  supported
a3ffb627-a915-4720-836c-5638bdfed9a7  llm_as_a_judge_metrics_provider-function  2024-07-17T20:04:19.002Z  python
00f607c9-63b2-4784-b20a-52b823d4e9ee  KS Test Provider Function                 2024-07-09T21:22:42.002Z 

,GUID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,16cd38f5-a80d-439e-b379-1fb518d82563,Human Rating Metrics Provider Function,2024-07-19T15:21:33.002Z,python,supported,
1,12cc063d-01bd-4a40-82cd-4db68624f24c,LLM Metrics Provider Function,2024-07-19T14:26:51.002Z,python,supported,
2,98345c71-f4b2-4fed-bbc7-e125374e5e4a,GenAI Metrics Provider Function,2024-07-19T13:38:36.002Z,python,supported,
3,760f0e2d-9c98-4716-a30f-a00477b62aee,test Metrics Provider Function,2024-07-19T03:51:08.002Z,python,supported,
4,a3ffb627-a915-4720-836c-5638bdfed9a7,llm_as_a_judge_metrics_provider-function,2024-07-17T20:04:19.002Z,python,,
5,00f607c9-63b2-4784-b20a-52b823d4e9ee,KS Test Provider Function,2024-07-09T21:22:42.002Z,python,supported,
6,eb1326eb-0a43-42b2-85ea-e74b318256a5,PSI Metric Provider Function,2024-07-09T21:13:36.002Z,python,supported,


### Create the function meta properties.


In [70]:
software_spec_id =  wml_client.software_specifications.get_id_by_name('runtime-23.1-py3.10')
print(software_spec_id)
function_meta_props = {
     wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
     wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_id
     }

336b29df-e0e1-5e7d-b6a5-f6ab722625b2


### Store the Python function.

In [71]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=custom_metrics_provider)
function_uid = wml_client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

Function UID = 76792d95-a5bd-4c2d-b682-6e99f0cfe10a


In [72]:
function_details = wml_client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

{'entity': {'software_spec': {'id': '336b29df-e0e1-5e7d-b6a5-f6ab722625b2',
                              'name': 'runtime-23.1-py3.10'},
            'type': 'python'},
 'metadata': {'created_at': '2024-07-19T16:16:19.686Z',
              'id': '76792d95-a5bd-4c2d-b682-6e99f0cfe10a',
              'modified_at': '2024-07-19T16:16:20.289Z',
              'name': 'LLM-as-a-Judge Metrics Provider Function',
              'owner': '1000331001',
              'space_id': 'ab5e72e2-80b1-4917-b0f8-b11b9c487c22'},
 'system': {'warnings': []}}


### Deploy the Python function.


In [73]:
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('S')
hardware_spec_id

'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b'

### Create deployment metadata for the Python function.

In [74]:
deploy_meta = {
 wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
 wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
 wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
}

### Create a deployment.

In [75]:
deployment_details = wml_client.deployments.create(function_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '76792d95-a5bd-4c2d-b682-6e99f0cfe10a' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='918d29a9-6a67-4761-b7dd-0c35d9f0767f'
------------------------------------------------------------------------------------------------




### Get the scoring URL.


In [76]:
created_at = deployment_details['metadata']['created_at']
find_string_pos = created_at.find("T")
if find_string_pos is not -1:
    current_date = created_at[0:find_string_pos]
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
scoring_url = scoring_url + "?version="+current_date
print(scoring_url)


https://cpd-cpd.scotiabank-watsonx-75a08960998068286ff1d6f41bddfabb-0000.ca-tor.containers.appdomain.cloud/ml/v4/deployments/918d29a9-6a67-4761-b7dd-0c35d9f0767f/predictions?version=2024-07-19


## 4. Configure OpenScale. <a name="config"></a>

Import the required libraries and set up the Watson OpenScale Python client.

In [77]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMetricRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MetricThreshold
from ibm_watson_openscale.supporting_classes.enums import MetricThresholdTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorTagRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Target
from ibm_watson_openscale.supporting_classes.enums import TargetTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems

from datetime import datetime, timezone, timedelta
import uuid

In [78]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        apikey=WOS_CREDENTIALS['apikey'],
        disable_ssl_verification=True
    )
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator, service_instance_id=WOS_GUID)
wos_client.version

'3.0.39'

In [79]:
wos_client.subscriptions.show(20)

20cfa2e2-8e1c-4e64-8879-046111bb73ac,prompt,[Knowledge transfer] [GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,a6b1d9ff-47c7-4ac3-b576-a66aece67dd4,[Knowledge transfer] [GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),6cacda08-9eab-4cd3-b37d-2e53162532c1,active,2024-07-18 15:14:50.258000+00:00,7d571243-a477-4ba4-9ec0-1f5d69a960ac
16e55c78-747e-448d-a4a7-7b4becf4920c,prompt,testing [GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,fc079586-bc4e-4230-851d-a62d27b1bd2b,testing [GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),6cacda08-9eab-4cd3-b37d-2e53162532c1,active,2024-07-18 02:35:07.255000+00:00,cfb549a9-125e-47c6-b7c2-2889fd5632e0
1c538fff-42b2-41b7-a154-9ad3aa31134f,prompt,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,7c05c2aa-4ed8-455d-b4a0-c486f331bbff,[demo][GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),22e3a00d-e7d2-4ce0-a01d-32eae862aa33,active,2024-07-17 20:04:26.842000+00:00,3bcdc97e-a635-4a45-8b2c-0cfcbddcac30
be541794-4374-443e-a069-6325d990a528,prompt,External prompt - Vertex Gemini 1.5,00000000-0000-0000-0000-000000000000,7a5958e8-8fcc-4af8-a973-5a0f2e716b8a,External prompt - Vertex Gemini 1.5,22e3a00d-e7d2-4ce0-a01d-32eae862aa33,active,2024-07-17 15:58:00.273000+00:00,9eaaca61-fe47-4637-891f-0c20dd441f68
c07bfc4a-e476-4a21-9066-babbcd2155f8,prompt,External GCP Prompt Sample(Vertex AI Gemini Flash),00000000-0000-0000-0000-000000000000,978cb533-fef9-4ef4-a581-6546309dbd4b,GCP Call Summarization (Vertex AI Gemini Flash),22e3a00d-e7d2-4ce0-a01d-32eae862aa33,active,2024-07-17 14:02:09.165000+00:00,3cdb5e3e-6ca0-4819-bdd9-820b212e7084
b4bcc11e-5fb9-428b-9937-0aaf1cee49a8,prompt,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,3834cf9b-e38d-4d13-82e8-ae22b1be2725,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro)-Prod,22e3a00d-e7d2-4ce0-a01d-32eae862aa33,active,2024-07-17 13:52:30.454000+00:00,e7a0964e-de89-49a3-b77e-9307ba21e170
05eaab54-760b-4f51-b272-9412699ae648,prompt,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,a2b8d2b7-b1af-412b-aa78-23f3e0d5bcb4,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro)-Pre-Prod,30446f24-a9dd-4b84-b80c-83dff76db8ed,active,2024-07-17 13:42:41.367000+00:00,3d7da07d-a369-4239-998c-70d4c2d8476e
7a0f7749-9849-4460-b783-e828a8c2ea2e,prompt,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),00000000-0000-0000-0000-000000000000,b33c6224-b69d-415d-b583-b65caac25c45,[GCP VertexAI] Call Center dialog Summerization (Gemini-Pro),6cacda08-9eab-4cd3-b37d-2e53162532c1,active,2024-07-17 13:33:31.073000+00:00,7bcc9016-9e6b-451c-b729-be7e0c6b19b8
832ebc15-e3f2-4aa4-9388-30001ccf0099,prompt,External GCP Prompt Sample(Vertex AI Gemini Flash),00000000-0000-0000-0000-000000000000,4dff4faa-7b6a-4e92-817c-0ff03ecc6958,External GCP Prompt Sample(Vertex AI Gemini Flash),6cacda08-9eab-4cd3-b37d-2e53162532c1,active,2024-07-15 21:10:59.372000+00:00,64df159c-90e1-4c43-8cd3-b71e20aeed92
26d57d90-cfaa-4ca0-ab92-686396b69d20,prompt,External prompt sample (Vertex AI Gemini Flash),00000000-0000-0000-0000-000000000000,d5be4e46-6e78-4b3e-8472-f4652f5aadd0,External prompt sample (Vertex AI Gemini Flash),6cacda08-9eab-4cd3-b37d-2e53162532c1,active,2024-07-15 21:02:35.525000+00:00,50ad7d15-eaa7-44d7-a49a-f2fde10e05f8
92d96b7b-1629-437b-8acd-3e85a6e7c889,prompt,Banking FAQ RAG prompt (GCP VertexAI),00000000-0000-0000-0000-000000000000,de17772e-0b05-492a-8d02-0cbd731f43af,Banking FAQ RAG prompt (GCP VertexAI) - Deploy,22e3a00d-e7d2-4ce0-a01d-32eae862aa33,active,2024-07-15 15:42:46.293000+00:00,357d109b-058e-434c-bf8f-04289c502ad6


## 5. Create the integrated system for the custom metrics provider. <a name="custom"></a>


Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics. 

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the `basic` authentication type, then provide `auth_type=basic` otherwise use `auth_type=bearer`.

In [80]:
auth_type = "bearer" #Supported values are basic and bearer

if auth_type == "basic":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"basic",
        "username":  "*****",# update the username here 
        "password": "*****"# Update the password here
   }
    
if auth_type == "bearer":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"bearer",
        "token_info": {
            "url": "{}/icp4d-api/v1/authorize".format(WOS_CREDENTIALS['url']),
            "headers": {
                "Content-Type": "application/json",
                "Accept": "application/json"
            },
            "payload": {
                "username": WOS_CREDENTIALS['username'],
                "api_key": WOS_CREDENTIALS['apikey'],
            },
            "method": "post"
        }
    }
    
  #if custom metrics deployment is on other cpd cluster or some other cloud then please uncomment and update 
  #the below "TOKEN_INFO" properties to generate the token to communicate to the custom metrics deployment url
  #Here are the sample values given in the token_info
    #TOKEN_INFO = {
    #    "url":  "https://iam.ng.bluemix.net/oidc/token", # update the token generation here 
    #    "headers": { "Content-type": "application/x-www-form-urlencoded" }, # update the headers here 
    #    "payload": "grant_type=urn:ibm:params:oauth:grant-type:apikey&response_type=cloud_iam&apikey=<api_key>", # update the payload here 
    #    "method": "POST" # # update the http method here 
    #}
    #CUSTOM_METRICS_PROVIDER_CREDENTIALS["token_info"] = TOKEN_INFO


### Remove existing integrated system 

In [81]:
# Delete existing custom metrics provider integrated systems if present
integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems
for system in integrated_systems:
    if system.entity.type == 'custom_metrics_provider' and system.entity.name == CUSTOM_METRICS_PROVIDER_NAME:
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system LLM-as-a-Judge Metrics Provider


In [82]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=CUSTOM_METRICS_PROVIDER_NAME,
    description=CUSTOM_METRICS_PROVIDER_NAME,
    type="custom_metrics_provider",
    credentials= CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": CUSTOM_METRICS_PROVIDER_NAME,
        "endpoint": scoring_url
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(custom_metrics_integrated_system)

{
  "metadata": {
    "id": "e3cff0e4-2d54-43da-9423-6ad97547c777",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:e3cff0e4-2d54-43da-9423-6ad97547c777",
    "url": "/v2/integrated_systems/e3cff0e4-2d54-43da-9423-6ad97547c777",
    "created_at": "2024-07-19T16:18:07.818000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "LLM-as-a-Judge Metrics Provider",
    "type": "custom_metrics_provider",
    "description": "LLM-as-a-Judge Metrics Provider",
    "credentials": {
      "secret_id": "7c361be7-9b29-47d2-930c-df475d67340b"
    },
    "connection": {
      "display_name": "LLM-as-a-Judge Metrics Provider",
      "endpoint": "https://cpd-cpd.scotiabank-watsonx-75a08960998068286ff1d6f41bddfabb-0000.ca-tor.containers.appdomain.cloud/ml/v4/deployments/918d29a9-6a67-4761-b7dd-0c35d9f0767f/predictions?version=2024-07-19"
    }
  }
}


## 6. Set up the custom monitor definition and instance. <a name="instance"></a>


### Check for the existence of the custom monitor definition.


In [83]:
def get_custom_monitor_definition():
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    for definition in monitor_definitions:
        if CUSTOM_MONITOR_NAME == definition.entity.name:
            return definition
    return None   

### Create the  custom monitor definition.

Update the custom metric names, threshold types (`LOWER_LIMIT`, `UPPER_LIMIT`) and default values as required. You can define the threshold type as lower limit, upper limit, or both.

In [84]:
###################################################################
# Update your custom monitor metrics names in the following field. Use the same metric names for creating the 
# monitor definition and publishing the metrics to Openscale in your python function
####################################################################
CUSTOM_MONITOR_METRICS_NAMES = ['human_rating']
#Update the tag values if you want to fetch the metrics by tags
TAGS= ['region']
TAG_DESCRIPTION =['customer geographical region'] 

In [85]:
#Update the Threshold types and default values of the metrics
def custom_metric_definitions():
    
    metrics = [MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[0],
                                    thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)])]
    
    #Comment the below tags code if there are no tags to be created
    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]
    
    return metrics, tags

In [86]:
def create_custom_monitor_definition():
    # check if the custom monitor definition already exists or not
    existing_definition = get_custom_monitor_definition()

    # if it does not exists, then create a new one.
    if existing_definition is None:
        metrics, tags = custom_metric_definitions()
        custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, metrics=metrics, tags=tags, background_mode=False).result
    else:
        # otherwise, send the existing definition
        custom_monitor_details = existing_definition
    return custom_monitor_details

In [87]:
custom_monitor_details = create_custom_monitor_definition()
custom_monitor_id = custom_monitor_details.metadata.id
custom_monitor_id

'llm_as_a_judge_performance'

### Check the existence of custom monitor instance.


In [88]:
def get_custom_monitor_instance(custom_monitor_id):
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = WOS_GUID, monitor_definition_id = custom_monitor_id, target_target_id = subscription_id).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None

In [89]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 60 #time in seconds <update the time here>

#Maximum number of records to consider during custom monitor run evaluation. Update max_records value as per the requirement
max_records = None
#Minimum number of records to consider during custom monitor run evaluation. Update min_records value as per the requirement
min_records = None

### Update the custom monitor instance.

In [90]:
def update_custom_monitor_instance(custom_monitor_instance_id):
    payload = [
     {
       "op": "replace",
       "path": "/parameters",
       "value": {
           "custom_metrics_provider_id": integrated_system_id,
           "custom_metrics_wait_time":   custom_metrics_wait_time 
       }
     }
    ]
    if max_records is not None:
        payload[0]["value"]["max_records"] = max_records    
    if min_records is not None:
        payload[0]["value"]["min_records"] = min_records 
        
    response = wos_client.monitor_instances.update(custom_monitor_instance_id, payload, update_metadata_only = True)
    result = response.result
    return result

### For the custom monitor definition, create a custom monitor instance.


In [91]:
def create_custom_monitor_instance(custom_monitor_id):
    # Check if an custom monitor instance already exists
    existing_monitor_instance = get_custom_monitor_instance(custom_monitor_id)

    # If it does not exist, then create one
    if existing_monitor_instance is None:
        target = Target(
                target_type=TargetTypes.SUBSCRIPTION,
                target_id=subscription_id
            )
        parameters = {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_wait_time":   custom_metrics_wait_time 
        }
        if max_records is not None:
            parameters["max_records"] =   max_records
        if min_records  is not None:
            parameters["min_records"] =   min_records 
        
        #Update your custom monitor metric ids in the below thresholds to update the default value
        thresholds = [ MetricThresholdOverride(metric_id=CUSTOM_MONITOR_METRICS_NAMES[0], type = MetricThresholdTypes.LOWER_LIMIT, value=0.6)
                     ]
        # create the custom monitor instance id here.
        custom_monitor_instance_details = wos_client.monitor_instances.create(
                    data_mart_id=WOS_GUID,
                    background_mode=False,
                    monitor_definition_id=custom_monitor_id,
                    target=target,
                    parameters=parameters,
                    thresholds = thresholds
        ).result
    else:
        # otherwise, update the existing one with latest integrated system details.
        instance_id = existing_monitor_instance.metadata.id
        custom_monitor_instance_details = update_custom_monitor_instance(instance_id)
    return custom_monitor_instance_details

In [92]:
monitor_instance_details = create_custom_monitor_instance(custom_monitor_id)
custom_monitor_instance_id = monitor_instance_details.metadata.id
print(monitor_instance_details)

{
  "metadata": {
    "id": "9f82debc-eec0-4dbc-8ea8-646c7ca0f918",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9f82debc-eec0-4dbc-8ea8-646c7ca0f918",
    "url": "/v2/monitor_instances/9f82debc-eec0-4dbc-8ea8-646c7ca0f918",
    "created_at": "2024-07-19T16:15:10.747000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-19T16:18:14.212000Z",
    "modified_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "llm_as_a_judge_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "357d109b-058e-434c-bf8f-04289c502ad6"
    },
    "parameters": {
      "custom_metrics_provider_id": "e3cff0e4-2d54-43da-9423-6ad97547c777",
      "custom_metrics_wait_time": 60
    },
    "thresholds": [
      {
        "metric_id": "llm_as_a_judge",
        "type": "lower_limit",
        "value": 0.6
      }
    ],
    "status": {
   

## Recap of the steps performed in this notebook

- Create a python function
- Deploy the python function to WML
- Create an OpenScale Integrated System pointing to the python function
- Create a Custom Monitor Definition mentioning various custom metrics
- Create a Custom Monitor Instance and specify the Integrated System ID in the monitor instance configuration.

### Upon publishing required payload logging or feedback logging data, please visit OpenScale console and run `Evaluate Now` from OpenScale Model Risk Management dashboard / Actions menu to evaluate the configured Custom Metrics Provider. For a production based subscriptions, the custom metrics provider would be invoked periodically.

# [OPTIONAL STEP] Invoke the custom metrics deployment Python function as part of this notebook.

Validate the custom metrics provider deployment by providing the correct set of paramaters to generate the custom metrics.

In [93]:
import uuid
parameters = {
    "custom_metrics_provider_id": integrated_system_id,
    "custom_metrics_wait_time":   custom_metrics_wait_time,
    "run_details": {
    "run_id": str(uuid.uuid4()),
    "run_status": "Running"
    }
}
if max_records is not None:
    parameters["max_records"] =   max_records
if min_records is not None:
    parameters["min_records"] =   min_records 
    
payload= {
    "data_mart_id" : WOS_GUID,
    "subscription_id" : subscription_id,
    "custom_monitor_id" : custom_monitor_id,
    "custom_monitor_instance_id" : custom_monitor_instance_id,
    "custom_monitor_instance_params": parameters
    
}

input_data= { "input_data": [ { "values": payload } ] }

In [94]:
deployment_uid=wml_client.deployments.get_uid(deployment_details)
job_details = wml_client.deployments.score(deployment_uid, input_data)
pprint(job_details)


{'predictions': [{'values': [[{'measurement_id': '6139c873-062c-4fc0-9441-f561153afa90',
                               'metrics': [{'llm_as_a_judge': 6.4,
                                            'region': 'us-south'}],
                               'run_id': '01b2ab4d-85fc-4810-8f98-ba745842c9b1',
                               'timestamp': '2024-07-19T16:18:20.196159Z'}]]}]}


# [OPTIONAL STEP] Test - Invoke the custom metrics deployment using Rest API

Simulation of Openscale invoking custom metrics provider deployment using Rest API 

In [95]:
import requests
def get_access_token():
    token_info = CUSTOM_METRICS_PROVIDER_CREDENTIALS["token_info"]
    url = token_info["url"]
    
    response = requests.post(url, headers=token_info["headers"], json=token_info["payload"], verify=False)
    json_data = response.json()
    access_token = json_data['token']
    return access_token


In [96]:
headers = dict()
headers["Content-Type"] = "application/json"
headers["Authorization"] = 'bearer '+get_access_token()

response = requests.post(scoring_url, json=input_data, headers=headers, verify=False)
json_data = response.json()
json_data

{'predictions': [{'values': [[{'measurement_id': '0b55f576-88f7-4a6f-8e56-ece3f2512638',
      'metrics': [{'llm_as_a_judge': 6.4, 'region': 'us-south'}],
      'run_id': '01b2ab4d-85fc-4810-8f98-ba745842c9b1',
      'timestamp': '2024-07-19T16:18:27.295232Z'}]]}]}

## Congratulations

You have finished configuring Custom Monitor Definition and Monitor instance for your Subscription. You can now run the custom monitor from `Watson OpenScale Dashboard`(https://url-to-your-cp4d-cluster/aiopenscale). Click the tile of your model and select `Evaluate Now` option from `Actions` drop down menu to run the monitor.